In [ ]:
!pip install geopy

In [1]:
import pandas as pd
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
import os
import time

# --- 1. Carica dataset completo ---
df = pd.read_csv("listings_palermo.csv")

# --- 2. Inizializza il geolocator ---
geolocator = Nominatim(user_agent="bnb_app_palermo")
reverse = RateLimiter(geolocator.reverse, min_delay_seconds=1)

# --- 3. Carica la cache (quartieri già trovati) ---
cache_file = "quartieri_cache.csv"
if os.path.exists(cache_file):
    cache_df = pd.read_csv(cache_file)
else:
    cache_df = pd.DataFrame(columns=["latitude", "longitude", "quartiere_geo"])

# --- 4. Funzione con cache ---
def get_quartiere_cached(lat, lon):
    lat = round(lat, 5)
    lon = round(lon, 5)

    match = cache_df[(cache_df["latitude"] == lat) & (cache_df["longitude"] == lon)]
    if not match.empty:
        return match.iloc[0]["quartiere_geo"]
    
    try:
        location = reverse((lat, lon), language="it")
        if location:
            addr = location.raw.get("address", {})
            quartiere = (
                addr.get("neighbourhood") or
                addr.get("suburb") or
                addr.get("city_district") or
                addr.get("municipality") or
                None
            )
        else:
            quartiere = None
    except:
        quartiere = None

    new_entry = pd.DataFrame([{
        "latitude": lat,
        "longitude": lon,
        "quartiere_geo": quartiere
    }])
    cache_df.loc[len(cache_df)] = new_entry.iloc[0]
    return quartiere

# --- 5. Carica file aggiornato se esiste ---
output_file = "listings_palermo_con_district.csv"
if os.path.exists(output_file):
    df = pd.read_csv(output_file)
else:
    df["district_area"] = None

# --- 6. Elabora solo le righe senza quartiere ---
rows_to_process = df[df["district_area"].isnull()]
print(f"Totale righe da processare: {len(rows_to_process)}")

# --- 7. Loop a blocchi ---
batch_size = 100  # puoi cambiare
for i, (index, row) in enumerate(rows_to_process.iterrows()):
    quartiere = get_quartiere_cached(row["latitude"], row["longitude"])
    df.at[index, "district_area"] = quartiere

    # Salvataggio ogni 10 righe
    if i % 10 == 0:
        df.to_csv(output_file, index=False)
        cache_df.to_csv(cache_file, index=False)
        print(f"Salvato dopo {i} righe...")

    # Stop dopo ogni batch per sicurezza (puoi togliere se vuoi farlo tutto in un colpo)
    if i > 0 and i % batch_size == 0:
        print(f"\n⏸️ Blocco di {batch_size} completato. Attendi 5 secondi...")
        df.to_csv(output_file, index=False)
        cache_df.to_csv(cache_file, index=False)
        time.sleep(5)

# --- 8. Salvataggio finale ---
df.to_csv(output_file, index=False)
cache_df.to_csv(cache_file, index=False)
print("✅ Completato!")

Totale righe da processare: 5401
Salvato dopo 0 righe...
Salvato dopo 10 righe...
Salvato dopo 20 righe...
Salvato dopo 30 righe...
Salvato dopo 40 righe...
Salvato dopo 50 righe...
Salvato dopo 60 righe...
Salvato dopo 70 righe...
Salvato dopo 80 righe...
Salvato dopo 90 righe...
Salvato dopo 100 righe...

⏸️ Blocco di 100 completato. Attendi 5 secondi...
Salvato dopo 110 righe...
Salvato dopo 120 righe...
Salvato dopo 130 righe...
Salvato dopo 140 righe...
Salvato dopo 150 righe...
Salvato dopo 160 righe...
Salvato dopo 170 righe...
Salvato dopo 180 righe...
Salvato dopo 190 righe...
Salvato dopo 200 righe...

⏸️ Blocco di 100 completato. Attendi 5 secondi...
Salvato dopo 210 righe...
Salvato dopo 220 righe...
Salvato dopo 230 righe...
Salvato dopo 240 righe...
Salvato dopo 250 righe...
Salvato dopo 260 righe...
Salvato dopo 270 righe...
Salvato dopo 280 righe...
Salvato dopo 290 righe...
Salvato dopo 300 righe...

⏸️ Blocco di 100 completato. Attendi 5 secondi...
Salvato dopo 310 ri

RateLimiter caught an error, retrying (0/2 tries). Called with (*((38.1095, 13.36057),), **{'language': 'it'}).
Traceback (most recent call last):
  File "\\?\C:\Users\andre\AppData\Roaming\jupyterlab-desktop\jlab_server\Lib\site-packages\urllib3\connectionpool.py", line 536, in _make_request
    response = conn.getresponse()
               ^^^^^^^^^^^^^^^^^^
  File "\\?\C:\Users\andre\AppData\Roaming\jupyterlab-desktop\jlab_server\Lib\site-packages\urllib3\connection.py", line 464, in getresponse
    httplib_response = super().getresponse()
                       ^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\andre\AppData\Roaming\jupyterlab-desktop\jlab_server\Lib\http\client.py", line 1428, in getresponse
    response.begin()
  File "C:\Users\andre\AppData\Roaming\jupyterlab-desktop\jlab_server\Lib\http\client.py", line 331, in begin
    version, status, reason = self._read_status()
                              ^^^^^^^^^^^^^^^^^^^
  File "C:\Users\andre\AppData\Roaming\jupyterlab-desktop\j

Salvato dopo 3170 righe...


RateLimiter caught an error, retrying (0/2 tries). Called with (*((38.11595, 13.35197),), **{'language': 'it'}).
Traceback (most recent call last):
  File "\\?\C:\Users\andre\AppData\Roaming\jupyterlab-desktop\jlab_server\Lib\site-packages\urllib3\connectionpool.py", line 536, in _make_request
    response = conn.getresponse()
               ^^^^^^^^^^^^^^^^^^
  File "\\?\C:\Users\andre\AppData\Roaming\jupyterlab-desktop\jlab_server\Lib\site-packages\urllib3\connection.py", line 464, in getresponse
    httplib_response = super().getresponse()
                       ^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\andre\AppData\Roaming\jupyterlab-desktop\jlab_server\Lib\http\client.py", line 1428, in getresponse
    response.begin()
  File "C:\Users\andre\AppData\Roaming\jupyterlab-desktop\jlab_server\Lib\http\client.py", line 331, in begin
    version, status, reason = self._read_status()
                              ^^^^^^^^^^^^^^^^^^^
  File "C:\Users\andre\AppData\Roaming\jupyterlab-desktop\

Salvato dopo 3180 righe...
Salvato dopo 3190 righe...


...

Salvato dopo 5040 righe...


RateLimiter caught an error, retrying (0/2 tries). Called with (*((38.11586, 13.35894),), **{'language': 'it'}).
Traceback (most recent call last):
  File "\\?\C:\Users\andre\AppData\Roaming\jupyterlab-desktop\jlab_server\Lib\site-packages\urllib3\connectionpool.py", line 536, in _make_request
    response = conn.getresponse()
               ^^^^^^^^^^^^^^^^^^
  File "\\?\C:\Users\andre\AppData\Roaming\jupyterlab-desktop\jlab_server\Lib\site-packages\urllib3\connection.py", line 464, in getresponse
    httplib_response = super().getresponse()
                       ^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\andre\AppData\Roaming\jupyterlab-desktop\jlab_server\Lib\http\client.py", line 1428, in getresponse
    response.begin()
  File "C:\Users\andre\AppData\Roaming\jupyterlab-desktop\jlab_server\Lib\http\client.py", line 331, in begin
    version, status, reason = self._read_status()
                              ^^^^^^^^^^^^^^^^^^^
  File "C:\Users\andre\AppData\Roaming\jupyterlab-desktop\

Salvato dopo 5050 righe...


RateLimiter caught an error, retrying (0/2 tries). Called with (*((38.13686, 13.35879),), **{'language': 'it'}).
Traceback (most recent call last):
  File "\\?\C:\Users\andre\AppData\Roaming\jupyterlab-desktop\jlab_server\Lib\site-packages\urllib3\connectionpool.py", line 536, in _make_request
    response = conn.getresponse()
               ^^^^^^^^^^^^^^^^^^
  File "\\?\C:\Users\andre\AppData\Roaming\jupyterlab-desktop\jlab_server\Lib\site-packages\urllib3\connection.py", line 464, in getresponse
    httplib_response = super().getresponse()
                       ^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\andre\AppData\Roaming\jupyterlab-desktop\jlab_server\Lib\http\client.py", line 1428, in getresponse
    response.begin()
  File "C:\Users\andre\AppData\Roaming\jupyterlab-desktop\jlab_server\Lib\http\client.py", line 331, in begin
    version, status, reason = self._read_status()
                              ^^^^^^^^^^^^^^^^^^^
  File "C:\Users\andre\AppData\Roaming\jupyterlab-desktop\

Salvato dopo 5060 righe...
Salvato dopo 5070 righe...
Salvato dopo 5080 righe...
Salvato dopo 5090 righe...
Salvato dopo 5100 righe...

⏸️ Blocco di 100 completato. Attendi 5 secondi...
Salvato dopo 5110 righe...
Salvato dopo 5120 righe...
Salvato dopo 5130 righe...
Salvato dopo 5140 righe...
Salvato dopo 5150 righe...
Salvato dopo 5160 righe...
Salvato dopo 5170 righe...
Salvato dopo 5180 righe...
Salvato dopo 5190 righe...
Salvato dopo 5200 righe...

⏸️ Blocco di 100 completato. Attendi 5 secondi...
Salvato dopo 5210 righe...
Salvato dopo 5220 righe...
Salvato dopo 5230 righe...
Salvato dopo 5240 righe...
Salvato dopo 5250 righe...
Salvato dopo 5260 righe...
Salvato dopo 5270 righe...
Salvato dopo 5280 righe...
Salvato dopo 5290 righe...
Salvato dopo 5300 righe...

⏸️ Blocco di 100 completato. Attendi 5 secondi...
Salvato dopo 5310 righe...
Salvato dopo 5320 righe...
Salvato dopo 5330 righe...
Salvato dopo 5340 righe...
Salvato dopo 5350 righe...
Salvato dopo 5360 righe...
Salvato do